In [ ]:
# Import libraries and functions

# Importing necessary libraries
import yfinance as yf
import pandas as pd

# Importing functions created in the "Financial_Ratios_Metrics.py" file
from Financial_Ratios_Metrics import (calculate_profit_margin, calculate_eps, calculate_dividends_per_share, 
                                      calculate_dividend_payout_ratio, calculate_roe, calculate_roa, calculate_roic, 
                                      calculate_interest_burden_ratio, calculate_interest_coverage_ratio, calculate_leverage_ratio,
                                      calculate_asset_turnover_ratio, calculate_fixed_asset_turnover_ratio, 
                                      calculate_inventory_turnover_ratio, calculate_receivables_turnover_ratio,
                                      calculate_payables_turnover_ratio, calculate_cash_conversion_cycle,
                                      calculate_current_ratio, calculate_quick_ratio, calculate_cash_ratio,
                                      calculate_market_to_book_ratio, calculate_pe_ratio, calculate_earnings_yield)






Chosen Stock: VEA.AX 
Last Share Price: 1.65 
Total Assets: 10603800000.0


In [ ]:
# Download the stock data

# Define the ticker for Viva Energy Group Limited
TICKER = "VEA.AX"

try:
    # Get stock information
    stock = yf.Ticker(TICKER)
    stock_prices = stock.history(period="10y")
    stock_info = stock.info
    stock_financials = stock.financials
    stock_balance_sheet = stock.balance_sheet

    # Extract relevant information
    last_price = stock_prices['Close'].iloc[-1]
    earnings = stock_financials.loc["Net Income"].iloc[0]
    dividend = stock_info['lastDividendValue']
    shares_outstanding = stock_info['sharesOutstanding']
    total_assets = (stock_balance_sheet.loc["Total Assets"].iloc[0] + stock_balance_sheet.loc["Total Assets"].iloc[1]) / 2

    print(f"Chosen Stock: {TICKER} \nLast Share Price: {last_price:.2f} \nTotal Assets: {total_assets}")
      
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Calculate financial ratios and metrics using functions from Financial_Ratios_Metrics.py

# Creating a dictionary with ratio names as keys and their respective functions as values (to calculate key ratios using imported functions)
ratios = {
    "Profit Margin": calculate_profit_margin(net_income, revenue),
    "EPS": calculate_eps(net_income, shares_outstanding),
    "DPS": calculate_dividends_per_share(dividends_paid, shares_outstanding),
    "DPR": calculate_dividend_payout_ratio(dividends_paid, net_income),
    "ROE": calculate_roe(net_income, total_equity),
    "ROA": calculate_roa(net_income, total_assets),
    "ROIC": calculate_roic(net_income, invested_capital),
    "Interest Burden": calculate_interest_burden_ratio(ebt, ebit),
    "Interest Coverage": calculate_interest_coverage_ratio(ebit, interest_expense),
    "Leverage": calculate_leverage_ratio(total_assets, total_equity),
    "Asset Turnover": calculate_asset_turnover_ratio(revenue, total_assets),
    "Fixed Asset Turnover": calculate_fixed_asset_turnover_ratio(revenue, net_fixed_assets),
    "Inventory Turnover": calculate_inventory_turnover_ratio(cost_of_goods_sold, average_inventory),
    "Receivables Turnover": calculate_receivables_turnover_ratio(revenue, average_receivables),
    "Payables Turnover": calculate_payables_turnover_ratio(cost_of_goods_sold, average_payables),
    "Cash Conversion Cycle": calculate_cash_conversion_cycle(average_inventory, average_receivables, average_payables),
    "Current Ratio": calculate_current_ratio(current_assets, current_liabilities),
    "Quick Ratio": calculate_quick_ratio(current_assets, inventory, current_liabilities),
    "Cash Ratio": calculate_cash_ratio(cash, current_liabilities),
    "Market-to-Book Ratio": calculate_market_to_book_ratio(market_cap, total_equity),
    "P/E Ratio": calculate_pe_ratio(last_price, eps),
    "Earnings Yield": calculate_earnings_yield(eps, last_price)
}

# Loop through the dictionary and print each ratio with its value
for ratio_name, ratio_value in ratios.items():
    # Format ratios with both real numbers and percentages to two decimal places
    print(f"{ratio_name}: {ratio_value:.2f}" if isinstance(ratio_value, float) else f"{ratio_name}: {ratio_value:.2%}")
